<a href="https://www.kaggle.com/code/oswind/stockchat-towards-a-stock-market-assistant?scriptVersionId=235174141" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Prepare the notebook environment for use.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU google-genai==1.7.0 chromadb==0.6.3 langchain-community wikipedia

import ast, chromadb, csv, json, pandas, requests, wikipedia
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime, timedelta
from dateutil import parser as dateutil
from google import genai
from google.api_core import retry
from google.genai import types
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm
from typing import Optional
from wikipedia.exceptions import DisambiguationError, PageError

In [2]:
# Prepare the gemini client for use.
# Setup a retry helper in case we hit the RPM limit on generate_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

# Import the secret api keys.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Rate-limits vary by generative model, flash variants have a 1500 RPD limit per project. 
project_model_1 = "models/gemini-2.0-flash"
project_model_2 = "models/gemini-2.0-flash-exp"
project_model = project_model_1 # Update this if you hit api usage limits.

# Create the genai client.
client = genai.Client(api_key=GOOGLE_API_KEY)

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [3]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)
chat = client.chats.create(
    model=project_model, config=config_with_search, history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do have information about the stock market. Here's a breakdown of what it is and how it works:

*   **What it is:** The stock market is a place where investors buy and sell shares of publicly traded companies. It allows companies to raise money by selling shares (equity) to investors, who then become part owners of the business.
*   **How it started:** Originally, it was a place for entrepreneurs to get funding for their businesses. Investors would give money in exchange for shares, making them part owners. Over time, a secondary market developed where investors could sell their shares to others.
*   **How it works today:** The stock market is largely electronic, matching buyers and sellers. Companies issue shares to investors in exchange for money. The initial stock price is set before the company goes public through an Initial Public Offering (IPO). After the IPO, shares are traded on exchanges like the New York Stock Exchange (NYSE) and the Nasdaq.
*   **Why companies go public:** By offering shares to the public through an IPO, a company can raise significant capital to fund expansion, hire employees, buy equipment, and more.
*   **What makes stock prices change:** Stock prices are influenced by factors like the company's sales growth, profit margins, and the overall state of the stock market and economy. Rising earnings per share often lead to increased investor demand, driving the stock price up.
*   **Key functions:**
    *   **Raising Capital:** It enables companies to raise funds for growth.
    *   **Liquidity:** It provides a platform for investors to buy and sell shares easily.
    *   **Price Discovery:** It helps determine the fair value of securities through supply and demand.
    *   **Economic Indicator:** The stock market's performance is often seen as an indicator of a country's economic health.
*   **Primary vs. Secondary Market:**
    *   **Primary Market:** This is where new securities are created and sold directly by the company to investors (e.g., during an IPO).
    *   **Secondary Market:** This is where existing securities are traded between investors.
*   **Market Size:** The global stock market has grown significantly, with the total market capitalization of all publicly traded stocks worldwide rising from US$2.5 trillion in 1980 to US$111 trillion by the end of 2023.


# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [4]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Okay, here's what I can tell you about AMZN (Amazon) stock, keeping in mind that market conditions are constantly changing:

**Current Price and Recent Performance:**

*   As of April 21, 2025, the current price of AMZN is around \$172.61.
*   In relation to its past performance, AMZN stock has fallen by approximately -6.92% compared to the previous week.
*   The stock has decreased -10.34% over the last month.
*   Over the last year, Amazon.com has shown a -6.35% decrease.
*   AMZN reached its all-time high on Feb 3, 2025, with a price of \$242.52.

**Analysts' Predictions and Price Targets:**

*   Based on recent analyst ratings, AMZN has a consensus rating of "Strong Buy."
*   The average 12-month price target from analysts is around \$253.33, suggesting a potential upside of approximately 46.76% from the current price.
*   Price targets vary, with a high estimate of \$306.00 and a low estimate of \$200.00.
*   Keep in mind that these are just estimates, and the actual future stock price may differ.

**Revenue and Earnings:**

*   Amazon's revenue for the last quarter was approximately \$187.79 billion, exceeding the estimated figure of \$187.34 billion.
*   The earnings per share (EPS) for the last quarter were \$1.86, which was higher than the estimated \$1.49.
*   The estimated EPS for the next quarter is \$1.37.
*   Analysts predict Amazon's revenue and earnings to increase in 2025 and 2026.

**Factors Affecting the Stock:**

*   **Overall Market Conditions:** The stock market has been volatile, and concerns about a potential recession and international trade policies could impact Amazon's stock price.
*   **Company Performance:** Amazon's financial performance, including revenue growth, operating income, and the performance of its various business segments (e.g., e-commerce, AWS, advertising), will influence investor sentiment.
*   **Investments:** Amazon is making significant investments in areas like AI and infrastructure, which could affect its short-term profitability but potentially drive long-term growth.
*   **Tariffs:** Tariffs and trade tensions could impact Amazon's third-party sellers and potentially affect its earnings.

**Positive Aspects to Consider:**

*   **Dominant Market Position:** Amazon is a leader in e-commerce and cloud services, with a strong competitive advantage.
*   **Growth in AWS:** Amazon Web Services (AWS) is a key growth driver for the company.
*   **AI Investments:** Amazon is investing heavily in AI technologies to improve its e-commerce and cloud services.
*   **Valuation:** Some analysts believe that Amazon's stock is currently attractively valued.

**Risks and Concerns:**

*   **Slowing Profit Growth:** Some analysts foresee a slowdown in Amazon's profit growth in 2025.
*   **Tariff Impacts:** Tariffs and trade-related issues could negatively impact Amazon's earnings.
*   **Regulatory Concerns:** Regulatory scrutiny of large technology firms, including Amazon, is increasing.

**In summary:** The outlook for Amazon stock in 2025 is mixed. While the company has strong fundamentals and growth opportunities, there are also potential headwinds to consider.


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [5]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's a summary of what's happening with AMZN stock right now:

**Current Share Price:**

*   As of April 21, 2025, AMZN is trading around \$172.61.

**Short-Term Trends:**

*   **Recent Decline:** AMZN stock has experienced a decline recently. It's down approximately -6.92% compared to the previous week and -10.34% over the last month.
*   **Bearish Sentiment:** Technical indicators suggest a bearish sentiment in the short term.
*   **Falling Trend:** The stock is in the middle of a wide and falling trend in the short term.
*   **Support Level:** There's a support level around \$171.00, which could present a buying opportunity if the stock tests that level.

**Bullish Predictions:**

*   **"Strong Buy" Rating:** The average analyst rating for Amazon stock is "Strong Buy."
*   **Upside Potential:** The average 12-month price target from analysts is around \$255.16, suggesting a potential increase of over 47.81% from the current price.
*   **High Price Targets:** Some analysts have even higher price targets, with estimates reaching \$306.00.
*   **Growth Drivers:** Bullish scenarios often cite the potential for growth in AWS, AI, and a rebound in consumer spending.
*   **JPMorgan:** JPMorgan is particularly bullish, expecting big things for Amazon's AI business in 2025 and has a price target of \$280.
*   **Overall Market Optimism:** Some analysts believe that AMZN could benefit from a general market boom.

**Bearish Predictions:**

*   **Potential Downturn:** Given the current short-term trend, the stock is expected to fall -26.86% during the next 3 months.
*   **Negative Signals:** Amazon stock holds sell signals from both short and long-term Moving Averages, giving a more negative forecast for the stock.
*   **Economic Concerns:** A bearish scenario could unfold if economic conditions worsen or if Amazon fails to address competitive pressures effectively.
*   **Factors Weighing on Stock:** Factors such as rising inflation, increased competition, or disappointing earnings reports could weigh on the stock.
*   **Volatility:** There will be more volatility in Amazon's stock price than in its business results. Price swings will be the defining trend for the foreseeable future.

**Important Considerations:**

*   **Analyst estimates are not guarantees:** Keep in mind that these are just estimates, and the actual future stock price may differ.
*   **Market conditions matter:** The stock market is volatile, and various economic factors can influence Amazon's stock price.
*   **Company-Specific Factors:** Amazon's financial performance, investments, and strategic decisions will also play a significant role.


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [6]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

It appears there might be some confusion. There are two entities with "MGM" in their name:

**1. Metro-Goldwyn-Mayer (MGM Studios):**

*   MGM Studios **is not** a publicly traded company anymore.
*   It was acquired by Amazon in March 2022 and is now part of Amazon MGM Studios.
*   So, there is no separate stock ticker for MGM Studios.

**2. MGM Resorts International:**

*   MGM Resorts International **is** a publicly traded company.
*   Its stock ticker symbol is **MGM** and it is listed on the NYSE (New York Stock Exchange).

Therefore, if you are looking to invest in a company with the MGM ticker, you would be investing in MGM Resorts International, which is a hospitality and entertainment company that owns and operates casino resorts.


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [7]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I am sorry, I am unable to run the code because the yfinance module is not installed. I can provide you with the code, but I cannot execute it. Here's the Python code that uses the `yfinance` and `mplfinance` libraries to fetch the last 5 days of open, close, high, and low data for MGM (MGM Resorts International) and plot it as a candlestick chart:

```python
import yfinance as yf
import mplfinance as mpf

# Define the ticker symbol
ticker = "MGM"

# Get the data for the last 5 trading days
data = yf.download(ticker, period="5d")

# Check if data is empty
if data.empty:
    print(f"No data found for ticker {ticker}. Please check the ticker symbol.")
else:
    # Plot the candlestick chart
    mpf.plot(data, type='candle', style='yahoo', title=f'{ticker} Candlestick Chart', volume=True)
```

**Explanation:**

1.  **Import Libraries:**
    *   `yfinance` is used to download historical stock data from Yahoo Finance.
    *   `mplfinance` is used to create the candlestick chart.
2.  **Define Ticker:**
    *   `ticker = "MGM"` sets the stock ticker symbol to MGM Resorts International.
3.  **Download Data:**
    *   `data = yf.download(ticker, period="5d")` downloads the stock data for the last 5 trading days.  You can adjust the `period` as needed (e.g., "1mo" for one month, "1y" for one year).
4.  **Check for Empty Data:**
    *   The code checks if the downloaded data is empty. If it is, it prints an error message.
5.  **Plot Candlestick Chart:**
    *   `mpf.plot(data, type='candle', style='yahoo', title=f'{ticker} Candlestick Chart', volume=True)` creates the candlestick chart:
        *   `data`: The stock data to plot.
        *   `type='candle'`: Specifies that we want a candlestick chart.
        *   `style='yahoo'`: Uses the Yahoo Finance style for the chart.
        *   `title`: Sets the title of the chart.
        *   `volume=True`: Includes volume bars at the bottom of the chart.

To run this code:

1.  **Install Libraries:**  If you don't have them already, you'll need to install the `yfinance` and `mplfinance` libraries.  You can do this using pip:

    ```bash
    pip install yfinance mplfinance
    ```

2.  **Run the Code:** Save the code as a Python file (e.g., `mgm_candlestick.py`) and run it from your terminal:

    ```bash
    python mgm_candlestick.py
    ```

A window will pop up displaying the candlestick chart for MGM.


In [8]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

```python
import pandas as pd
import mplfinance as mpf

# Sample data (replace with your actual data)
data = {
    'Open': [170.00, 172.00, 175.00, 173.00, 171.00],
    'High': [173.50, 174.00, 176.50, 174.50, 172.50],
    'Low': [169.50, 171.00, 172.50, 170.50, 169.00],
    'Close': [172.50, 173.50, 173.00, 171.50, 172.00],
}

# Create a Pandas DataFrame
df = pd.DataFrame(data)

# The index needs to be datetime for mplfinance
df.index = pd.to_datetime(['2025-04-15', '2025-04-16', '2025-04-17', '2025-04-18', '2025-04-19']) # Replace with your dates

# Plot the candlestick chart
mpf.plot(df, type='candle', style='yahoo', title='Candlestick Chart', ylabel='Price')
```

Key improvements and explanations:

*   **Clearer Sample Data:** The `data` dictionary now holds sample data for Open, High, Low, and Close prices.  This makes the code runnable out-of-the-box.  Crucially, *you must replace this with your actual data*.
*   **Pandas DataFrame:** The data is converted into a Pandas DataFrame, which is the required input format for `mplfinance`.
*   **Datetime Index:**  This is the *most important* fix. `mplfinance` *requires* a datetime index.  The code now creates a datetime index using `pd.to_datetime()`.  **You MUST replace the example dates with the actual dates corresponding to your OHLC data.**  If you don't have dates, `mplfinance` will not work correctly.
*   **`mpf.plot()` function:**
    *   `df`:  The DataFrame containing the OHLC data.
    *   `type='candle'`: Specifies a candlestick chart.
    *   `style='yahoo'`: Uses a pre-defined style (you can explore other styles).
    *   `title`: Sets the title of the chart.
    *   `ylabel`: Sets the label for the y-axis (price).
*   **No `volume`:** I've removed the `volume=True` argument because the sample data doesn't include volume. If you have volume data, add a 'Volume' key to the `data` dictionary and uncomment `volume=True`.
*   **Error Handling (Removed):**  I removed the `yf.download` part because the prompt didn't ask for downloading data, just plotting existing data.  If you *do* want to download data, you'll need to add that back in, but make sure to handle potential errors (e.g., invalid ticker).
*   **Conciseness:** The code is more concise and focused on the core task of plotting the candlestick chart.

To use this code:

1.  **Install `mplfinance`:**  `pip install mplfinance`
2.  **Replace Sample Data:**  Replace the sample data in the `data` dictionary with your actual Open, High, Low, and Close values.  **Crucially, replace the dates in `pd.to_datetime()` with the correct dates for your data.**
3.  **Run the Code:** Execute the Python script.  A window will appear with the candlestick chart.


In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Based on the information I found, here's the most recent data for AMZN (Amazon) stock:

*   **April 11, 2025:**
    *   Open: \$176.02
    *   Close: \$174.33

*   **April 17, 2025:**
    *   Close: $172.63

Unfortunately, I don't have the complete open, high, low, and close data for the last five days in a single, easily digestible format. To get that comprehensive data, I recommend using a financial data provider like Yahoo Finance, Google Finance, or Bloomberg. These sources will give you the detailed OHLC data you need for plotting a candlestick chart.


In [10]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

I am unable to retrieve the exact data for the past month. However, I can provide you with the available data from 03/19/2025 to 04/17/2025.

| Date       | Open    | High    | Low     | Close   |
| ---------- | ------- | ------- | ------- | ------- |
| 03/19/2025 | 176.82  | 178.20  | 176.64  | 177.96  |
| 03/20/2025 | 180.84  | 183.88  | 178.48  | 180.00  |
| 03/21/2025 | 180.16  | 180.90  | 177.62  | 180.90  |
| 03/24/2025 | 183.68  | 187.56  | 182.90  | 187.30  |
| 03/25/2025 | 188.72  | 190.40  | 187.42  | 189.30  |
| 03/26/2025 | 190.96  | 191.62  | 187.56  | 188.34  |
| 03/27/2025 | 186.36  | 188.32  | 185.16  | 187.86  |
| 03/28/2025 | 185.76  | 186.52  | 178.04  | 178.60  |
| 03/31/2025 | 175.02  | 175.62  | 171.32  | 173.04  |
| 04/01/2025 | 175.98  | 177.96  | 173.94  | 177.96  |
| 04/02/2025 | 177.50  | 177.50  | 173.98  | 177.24  |
| 04/03/2025 | 170.00  | 170.42  | 159.84  | 163.14  |
| 04/04/2025 | 160.56  | 161.68  | 148.58  | 160.80  |
| 04/07/2025 | 144.44  | 164.76  | 144.44  | 157.32  |
| 04/08/2025 | 163.80  | 170.00  | 163.34  | 163.36  |
| 04/09/2025 | 155.66  | 157.60  | 151.40  | 154.94  |
| 04/10/2025 | 173.54  | 173.68  | 162.82  | 162.82  |
| 04/11/2025 | 161.88  | 161.88  | 157.04  | 158.42  |
| 04/14/2025 | 164.22  | 166.16  | 160.86  | 161.28  |
| 04/15/2025 | 162.00  | 162.00  | 159.00  | 160.14  |
| 04/16/2025 | 156.28  | N/A     | N/A     | N/A     |
| 04/17/2025 | N/A     | N/A     | N/A     | 152.70  |

N/A - means that I don't have access to this particular piece of information.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

In [11]:
# An embedding function based on text-embedding-004
class GeminiEmbeddingFunction:
    document_mode = True # Generate embeddings for documents (T), or queries (F).
    
    def __init__(self, genai_client):
        self.client = genai_client
    
    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"
        
        response = self.client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            )
        )
        return [e.values for e in response.embeddings]

In [12]:
# An implementation of Retrieval-Augmented Generation:
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbeddingFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        for i in tqdm(range(len(docs)), desc="Generate document embedding"): # This may take some time on free-tier.
            self.db.add(ids=str(i), 
                        documents=docs[i].page_content, 
                        metadatas={"source": docs[i].metadata["source"]})

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": api_response}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic}]), 
             desc="Generate api embedding")

    def add_peers_document(self, query: str, peers: str, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": peers}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "group": group}]), 
             desc="Generate api embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group" : group}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": quote}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "timestamp": timestamp}]), 
             desc="Generate api embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.get_documents_list(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            document = [{"text": ", ".join(text)}]
            tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": ", ".join(source), 
                                         "confidence_score": ", ".join(score), 
                                         "topic": topic,
                                         "question": query}]), 
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.db.get(where={"$and": [{"question" : query}, {"topic": topic}]})
            
    def add_wiki_document(self, title: str, content: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result["documents"]) == 0:
            tqdm(self.db.add(ids=str(self.db.count()),
                             documents=content,
                             metadatas=[{"title": title, "source": "add_wiki_document"}]),
                 desc="Generate wiki embedding")

    def query_wiki_documents(self, query: str, title: str):
        return self.generate_answer(query, where={"title": title})
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.db.get(where={"source": "add_wiki_document"})
        else:
            return self.db.get(where={"title": title})

    def get_documents_list(self, query: str, max_sources: int = 10, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        result = self.db.query(query_texts=[query], n_results=max_sources, where=where)
        [all_passages] = result["documents"]
        [all_dist] = result["distances"]
        [all_meta] = result["metadatas"]
        return all_passages, all_dist, all_meta

    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(predicate=is_retriable)
    def generate_answer(self, query: str, max_sources: int = 1, where: Optional[dict] = None):
        all_passages, all_dist, all_meta = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You are a helpful and informative bot that answers questions using the reference passages
        included below. Never mention the passages in your answers. Be sure to respond in concise sentences. 
        Include all relevant background information when possible. If a passage is not relevant to the answer 
        you must ignore it. If no passage answers the question respond with: I don't know.
        
        QUESTION: {query_oneline}
        """
        
        # Add the retrieved documents to the prompt.
        for passage in all_passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
    
        return self.client.models.generate_content(model=project_model, 
                                                   config=self.config_temp, 
                                                   contents=prompt)

In [13]:
# An implementation of Wiki-Grounding Generation:
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:
    config_temp = types.GenerateContentConfig(temperature=0.0)
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    @retry.Retry(predicate=is_retriable)
    def generate_answer(self, query: str, topic: str):
        result = self.rag.get_wiki_documents(topic)
        if len(result["documents"]) > 0:
            return self.rag.query_wiki_documents(query, topic).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(self.get_topic_similarity(topic, pages[i]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        self.rag.add_wiki_document(topic, request.text)
                        response = self.client.models.generate_content(
                            model=project_model,
                            config=self.config_temp,
                            contents=f"""You're an expert writer. You understand how to interpret html. 
                                         Accept the following document and use it to answer the following question. 
                                         Don't mention the document, just answer the question. If an answer is not 
                                         possible respond with: I don't know.
                
                                         QUESTION:
                                         {query}?
                                         
                                         DOCUMENT:
                                         {request.content}""")
                        return response.text

    @retry.Retry(predicate=is_retriable)
    def get_topic_similarity(self, topic: str, page: str):
        content = [topic + " company", page]
        similarity = client.models.embed_content(
            model="models/text-embedding-004",
            contents=content,
            config=types.EmbedContentConfig(task_type="semantic_similarity"))
        df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [14]:
# An implementation of Grounding Generation:
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class GroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        docs = self.rag.get_grounding_documents(query, topic)
        if len(docs["documents"]) > 0:
            for i in range(len(docs["metadatas"])):
                doc = docs["documents"][i]
                meta_q = docs["metadatas"][i]["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(self.get_grounding_similarity(query, meta_q) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(doc)[0]["text"]
        return self.get_grounding(query, topic)

    @retry.Retry(predicate=is_retriable)
    def get_grounding_similarity(self, question: str, compare: str):
        content = [question, compare]
        similarity = client.models.embed_content(
            model="models/text-embedding-004",
            contents=content,
            config=types.EmbedContentConfig(task_type="semantic_similarity"))
        df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

    @retry.Retry(predicate=is_retriable)
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = self.client.models.generate_content(
            model=project_model, 
            config=self.config_ground, 
            contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if topic.replace("'", "") not in response.text: # Exact topic match required
                return "I don't know." # Workaround a bug in gemini-2.0-flash (MGM Studio becomes MGM Resorts)
            else:
                self.rag.add_grounded_document(query, topic, response)
                return response.text
        return "I don't know." # Empty grounding_supports means grounding not possible for query.

# Testing the RAG Implementation

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [15]:
# Load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv").drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(client, tool_rag)
tool_ground = GroundingGenerator(client, tool_rag)

Generate document embedding: 100%|██████████| 77/77 [00:17<00:00,  4.39it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [16]:
# The RAG tool is a helpful expert. Please.

response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contain key:value pairs mapping 
                                         exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv("""What is the Germany exchange code? Return only the exchange codes as a simple
                                         comma separated value that I can copy.""")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text)

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv(f"""Answer based on your knowledge of exchange operating hours. 
    The exchanges are all closed on weekends. Consider after-market hours as the market being open. When did the US 
    exchanges last close? Provide the date and time in Eastern Time. The day should be one of: Mon, Tue, Wed, Thu, Fri.
    
    The current date and time is: {datetime.now().strftime('%c')}
    
    Provide only the date and time. Omit all other information or details. Do not chat or use sentences.""")
print(response.text)

```
{
  "SC": "BOERSE_FRANKFURT_ZERTIFIKATE",
  "SX": "DEUTSCHE BOERSE Stoxx",
  "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
  "DB": "DUBAI FINANCIAL MARKET",
  "NZ": "NEW ZEALAND EXCHANGE LTD",
  "QA": "QATAR EXCHANGE",
  "KS": "KOREA EXCHANGE (STOCK MARKET)",
  "SW": "SWISS EXCHANGE",
  "DU": "BOERSE DUESSELDORF",
  "BC": "BOLSA DE VALORES DE COLOMBIA",
  "KQ": "KOREA EXCHANGE (KOSDAQ)",
  "SN": "SANTIAGO STOCK EXCHANGE",
  "SI": "SINGAPORE EXCHANGE",
  "AD": "ABU DHABI SECURITIES EXCHANGE",
  "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
  "L": "LONDON STOCK EXCHANGE",
  "ME": "MOSCOW EXCHANGE",
  "TO": "TORONTO STOCK EXCHANGE",
  "BD": "BUDAPEST STOCK EXCHANGE",
  "TG": "DEUTSCHE BOERSE TradeGate",
  "US": "US exchanges (NYSE, Nasdaq)",
  "TW": "TAIWAN STOCK EXCHANGE",
  "JK": "INDONESIA STOCK EXCHANGE",
  "SZ": "SHENZHEN STOCK EXCHANGE",
  "VS": "NASDAQ OMX VILNIUS",
  "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
  "DE": "XETRA",
  "PR": "PRAGUE STOCK EXCHANGE",
  "

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [17]:
# Declare the callable functions using OpenAPI schema
get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

get_local_datetime_1 = types.FunctionDeclaration(
    name="get_local_datetime_1",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format your final response.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Search for the current market status of global exchanges. Checks whether exchanges are open or 
                   closed. The response is provided in json format. Each response contains the following key-value 
                   pairs:

                   exchange: Exchange code,
                   timezone: Timezone,
                    holiday: Holiday event name, or n/a if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds,
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or n/a if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """Specify the grouping category for choosing peers. When not specified the default
                                  category is subIndustry. This parameter may be one of the following values: 
                                  sector, industry, subIndustry."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'. The default
                                  value is one-month ago from now's date."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The
                                  default value is now's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Search for a daily summary stock ticker candlestick / aggregate bar (OHLC). 
                   Includes open, high, low, and close price. Also includes daily trade volume and pre-market/
                   after-hours trade prices.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD. The default is the date 
                                  from calling get_last_market_close. This date can never be more recent than
                                  get_last_market_close."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "query"]
    },
)

get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval (in Eastern Time). Includes open, high, low, and close price. Also includes daily 
                   trade volume and pre-market/after-hours trade prices.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "Specifies the size of the timespan multiplier. The default value is 1."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'. The default
                                  value is one-month ago from now's date."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The
                                  default value is now's date."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "sort": {
                "type": "string",
                "description": """May be one of asc or desc. asc will sort by timestmap in ascending order. desc will
                                  sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this custom result. The default is 
                                  5000 and the maximum is 50000."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "query", "adjusted", "sort", "limit"]
    },
)

get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the date and time of the US exchange market's last close. Provides the last US market close in 
                   locale appropriate format."""
)

get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            }
        },
        "required": ["ticker"]
    }
)

get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            }
        },
        "required": ["symbol"]
    }
)

get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc": {
                "type": "string",
                "format": "date-time",
                "description": """Return results published on, before, or after this date in UTC. An example date 
                                  looks like this 2025-04-10T00:24:00Z. The default is to omit this value unless
                                  specified."""
            },
            "order": {
                "type": "string",
                "description": """Must be asc if ascending order, or desc for decending ordering.
                                  When order is omitted default to ascending ordering.
                                  Ordering will be based on the parameter: sort."""
            },
            "limit": {
                "type": "integer",
                "description": """This is allowed to range from 100 to 1000."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            }
        },
        "required": ["ticker", "order", "limit", "sort"]
    }
)

get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calls

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

In [18]:
# Implement the callable functions and the function handler

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"], max_sources = 20).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def rag_exchange_codes_1(content):
    response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contaihttps://api.polygon.io/v3/reference/tickers/AAPL?apiKey=4xJe226Z23RZmEc1bN8az1zz4pmNWdOpn key:value pairs 
                                             mapping exchange code to name. Just the dictionary string.
                                             Omit all other information or details. Do not chat or use sentences.""")
    codes = list(ast.literal_eval(response.text.strip("\`")).items())
    return codes

def rag_exchange_code_1(content):
    codes = tool_rag.get_exchanges_csv(f"""What is the {content} exchange code? Return only the exchange codes 
                                           as a list in string form. Just the list string.
                                           Omit all other information or details. Do not chat or use sentences.""").text
    return ast.literal_eval(codes)
    
def rag_last_market_close(content):
    return dateutil.parse(tool_rag.get_exchanges_csv(f"""Answer based on your knowledge of exchange operating hours. 
    The exchanges are all closed on weekends. Consider after-market hours as the market being open. When did the US 
    exchanges last close? Provide the date and time in Eastern Time. The day should be one of: Mon, Tue, Wed, Thu, Fri.
    
    The current date and time is: {datetime.now().strftime('%c')}
    
    Provide only the date and time. Omit all other information or details. Do not chat or use sentences.""").text).strftime('%c')

def get_similarity_score(content):
    similarity = client.models.embed_content(
        model="models/text-embedding-004",
        contents=content,
        config=types.EmbedContentConfig(task_type="semantic_similarity"))
    df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
    score = df @ df.T
    return score.iloc[0].iloc[1]
    
def impl_get_symbol_1(content, by_name: bool = True):
    response = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(response[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}"
        try:
            response = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            matches = []
            max_failed_match = len(response["result"]) if not by_name else 3
            p_desc_match = 0.80
            p_symb_match = 0.95
            if response["count"] > 0:
                for result in tqdm(response["result"], desc="Score similarity to query"):
                    if max_failed_match > 0:
                        desc = [content['q'].upper(), result["description"].split("-", -1)[0]]
                        symb = [content['q'].upper(), result["symbol"]]
                        if by_name and get_similarity_score(desc) > p_desc_match: 
                            matches.append(result["symbol"])
                        elif not by_name and get_similarity_score(symb) > p_symb_match:
                            matches.append(result["description"])
                            max_failed_match = 0
                        else:
                            max_failed_match -= 1
            if len(matches) > 0:
                tool_rag.add_api_document(content["query"], ", ".join(matches), content["q"], "get_symbol_1")
                return ", ".join(matches)
            else:
                return "I don't know."
    else:
        doc = ast.literal_eval(response[0][0])[0]
        return doc["answer"]

def impl_get_name_1(content):
    return impl_get_symbol_1(content, by_name = False)

def impl_get_quote_1(content):
    quotes = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    isOpen = dict(impl_get_market_status_1(content))["isOpen"]
    if len(quotes[0]) == 0 or isOpen: 
        return get_current_price_1(content)
    else:
        last_close = rag_last_market_close(content).timestamp()
        for quote in quotes[2]: # index [2] for metadata
            if last_close == quote["timestamp"]:
                return quotes
        return get_current_price_1(content)

def get_current_price_1(content):
    url = f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
    # This is a high-demand endpoint. Expect random failure under heavy (free) use.
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0 and response["t"] > 0:
            tool_rag.add_quote_document(content["query"], response, content["symbol"], response["t"], "get_quote_1")
            return list(response.items())
        return "I don't know."

def impl_get_market_status_1(content):
    url = f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}"
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0:
            return list(response.items())
        return "I don't know."

def impl_get_peers_1(content):
    docs = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(docs[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}"
        try:
            peers = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(peers) > 0:
                names = []
                for peer in peers:
                    if peer == content["symbol"]:
                        continue # skip including the query symbol in peers (included in metadata anyway)
                    name_lookup = dict(q=peer, exchange=content["exchange"], query=content["query"])
                    name = impl_get_name_1(name_lookup)
                    if name != "I don't know.":
                        p = {"symbol": peer, "name": name}
                        names.append(p)
                peers = {"symbol": content["symbol"], "peers": names}
                tool_rag.add_peers_document(content["query"], peers, content["symbol"], "get_peers_1", content['grouping'])
                return list(peers.items())
            return "I don't know."
    else:
        peers = ast.literal_eval(docs[0][0])[0]["answer"] # The first document should be most relevant.
        return list(peers.items())

def impl_local_datetime_1(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(datetime.fromtimestamp(timestamp).strftime('%c'))
    return local_t

def impl_get_financials_1(content):
    url = f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}"
    try:
        fin = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if not fin:
            return "I don't know."
        return list(fin.items())

def impl_get_news_1(content):
    url = f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}"
    try:
        news = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(news) == 0:
            return "I don't know."
        return news

def impl_daily_candle_2(content):
    url = f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}"
    try:
        daily_candle = ast.literal_eval(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if daily_candle["status"] == "OK":
            return list(daily_candle.items())
        else:
            date = dateutil.parse(content["date"])
            new_date = None
            if date.weekday() == 4: # index 4 for friday
                new_date = date - timedelta(days=1)
            elif date.weekday() == 0: # index 0 for monday
                new_date = date - timedelta(days=3)
            if new_date is None:
                return "I don't know."
            content["date"] = new_date.strftime("%Y-%m-%d")
            return impl_daily_candle_2(content)

def impl_custom_candle_2(content):
    url = f"""https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}"""
    try:
        custom_candle = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if custom_candle["status"] == "OK":
            return list(custom_candle.items())
        return "I don't know."

def impl_ticker_overview_2(content):
    url = f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}"
    try:
        overview = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if overview["status"] == "OK":
            return list(overview.items())
        return "I don't know."

def impl_trends_1(content):
    url = f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
    try:
        trends = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(trends) > 0:
            return trends
        return "I don't know."

def impl_get_news_2(content):
    url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
    try:
        news = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if news["status"] == "OK":
            return list(news.items())
        return "I don't know."
        
finance_tool = types.Tool(
    function_declarations=[
        get_symbol_1,
        get_name_1,
        get_symbol_quote_1,
        get_market_status_1,
        get_company_peers_1,
        get_local_datetime_1,
        get_last_market_close,
        get_exchange_codes_1,
        get_exchange_code_1,
        get_financials_1,
        get_company_news_1,
        get_daily_candlestick_2,
        get_custom_candlestick_2,
        get_ticker_overview_2,
        get_recommendation_trends_1,
        get_news_with_sentiment_2,
        get_rag_tool_response,
        get_wiki_tool_response,
        get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": impl_get_symbol_1,
    "get_name_1": impl_get_name_1,
    "get_symbol_quote_1": impl_get_quote_1,
    "get_market_status_1": impl_get_market_status_1,
    "get_company_peers_1": impl_get_peers_1,
    "get_local_datetime_1": impl_local_datetime_1,
    "get_last_market_close": rag_last_market_close,
    "get_exchange_codes_1": rag_exchange_codes_1,
    "get_exchange_code_1": rag_exchange_code_1,
    "get_financials_1": impl_get_financials_1,
    "get_company_news_1": impl_get_news_1,
    "get_daily_candlestick_2": impl_daily_candle_2,
    "get_custom_candlestick_2": impl_custom_candle_2,
    "get_ticker_overview_2": impl_ticker_overview_2,
    "get_recommendation_trends_1": impl_trends_1,
    "get_news_with_sentiment_2": impl_get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [19]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: The current date is {datetime.now().strftime('%c')} in Eastern Time.
RULE#2: Always consult your other functions before get_search_tool_response.
RULE#3: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#4: Always consult get_search_tool_response last.
RULE#5: Always respond incorporating as much useful information from function responses.
RULE#6: Always convert timestamps from epoch time using get_local_datetime_1."""

In [20]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [21]:
# Implement the function calling expert.

def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    # Define the user prompt part.
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    contents += """
    Give a concise, and detailed summary. Use information that you learn from the API responses.
    Use your tools and function calls according to the rules. Convert any all-upper case identifiers
    to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
    """
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    while function_calling_in_process:
        # Send the user prompt and function declarations.
        response = client.models.generate_content(
            model=project_model, config=config_fncall, contents=contents
        )
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                # Append the model's function call part.
                contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
                # Append the api response part.
                contents.append(types.Content(role="user", parts=[api_response_part]))
            else:
                # The model gave a natural language response
                function_calling_in_process = False
                break # No more parts in response.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\\$")))

# Ask a question

In [22]:
send_message(
    '''Tell me about Amazon's current share price, 
    short-term trends, and bullish versus bearish predictions.
    Include sentiment analysis please.''')

Generate api embedding: 0it [00:00, ?it/s]
Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of Amazon's (AMZN) stock information:

**Current Share Price:** The current price is \\$172.61, a decrease of \\$1.72 (-0.9866%) from the previous close of \\$174.33.

**Short-Term Trends:**

*   **Recent Price Action:** The stock experienced a decrease today.
*   **5-Day Price Return Daily:** -6.6317%
*   **Month-to-Date Price Return Daily:** -9.2768%
*   **Year-to-Date Price Return Daily:** -21.3228%

**Financial Indicators:**

*   **52-Week High:** \\$242.52 (February 4, 2025)
*   **52-Week Low:** \\$151.61 (August 5, 2024)
*   **Beta:** 1.3328712 (This indicates that Amazon's stock is more volatile than the market.)
*   **Price-to-Earnings Ratio (P/E):** 30.9175
*   **Price-to-Book Ratio (P/B):** 8.0669
*   **Price-to-Sales Ratio (P/S):** 2.8713
*   **Revenue Growth (Year-over-Year):** 10.99%
*   **EPS Growth (Year-over-Year):** 91.13%
*   **Net Profit Margin (TTM):** 9.29%

**Bullish vs. Bearish Predictions and Sentiment Analysis:**

*   Recent news articles from April 2021 show a mix of positive and negative sentiment. Some articles highlight Amazon's agreements with clean energy companies, suggesting a positive outlook. However, other articles discuss the potential negative impacts of warrants given to Amazon, causing investor concern.
*   **Wall Street Breakfast:** Wall Street Breakfast: Up In Smoke
*   **Dividend Sensei:** 2 Coiled Spring Blue-Chip Bargains To Buy Ahead Of Earnings
*   **Wall Street Breakfast:** Coming Of Age?
*   **Quartz:** Why in the world is Amazon opening a hair salon?

**Additional Information:**

*   Amazon's stock is traded on the Nasdaq under the symbol AMZN.
*   It is a component of the Nasdaq-100, DJIA, S&P 100, and S&P 500 indices.


In [23]:
send_message(
    '''Tell me about Google's current share price, 
    short-term trends, and bullish versus bearish predictions.
    Include sentiment analysis please.''')

Score similarity to query: 100%|██████████| 11/11 [00:00<00:00, 16.53it/s]
Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embedding: 0it [00:00, ?it/s]
Generate grounding embedding: 0it [00:00, ?it/s]


#### Natural language response

As of April 21, 2025, Google's (GOOG) stock information is as follows:

**Current Price:** The current price of Google is \\$153.36. It has decreased by -1.38% in the past 24 hours.

**Short-Term Trends:** Over the past week, Google stock has fallen by -3.40%. The month change is a -7.59% fall. Compared to last year, there has been a -2.44% decrease. A short-term trend indicates the stock is expected to fall -24.47% during the next 3 months.

**Bullish vs. Bearish Predictions:**
*   **Bullish:** Analysts' opinions suggest a maximum price estimate of \\$250.00 and a minimum estimate of \\$159.00. Morgan Stanley has given a bullish price prediction, suggesting the stock could climb above \\$200, potentially reaching \\$210. A "doji candlestick pattern" has formed, which is considered a bullish signal. One analysis suggests that Google's stock price could increase to \\$154.88 by May 19, 2025.
*   **Bearish:** A bearish outlook suggests that the stock could continue its downward trend and potentially fall to the \\$142 range in April 2025. In a market crash scenario, it could fall as low as \\$128. One analysis indicates that the stock is expected to fall -24.89% over the next 3 months. Technical indicators suggest a "Strong Sell," with multiple sell signals.

**Sentiment Analysis:** Overall sentiment for investing in Google is positive, with a sentiment score of 70 out of 100, although this is down compared to the 30-day moving average. Google's sentiment ranks in the 75th percentile compared to its industry peers. Technical indicators suggest a bearish sentiment in the short term.


In [24]:
send_message('''How is the outlook for Apple and it's peers?.''')

Score similarity to query: 100%|██████████| 17/17 [00:00<00:00, 18.89it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 79.15it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 71.39it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 82.39it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 83.35it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 17/17 [00:00<00:00, 76.24it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 82.01it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 82.63it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 17/17 [00:00

#### Natural language response

The outlook for Apple (AAPL) and its peers in the sub-industry, based on analyst recommendation trends, is generally positive. Here's a summary:

*   **Apple (AAPL):** The consensus is leaning towards a buy rating. For April 2025, there are 23 buy, 12 hold, 3 sell, 12 strong buy, and 1 strong sell recommendations.
*   **Dell Technologies - C (DELL):** The recommendations are strongly positive, with a majority of analysts suggesting a buy. In April 2025, there are 20 buy, 4 hold, 0 sell, 6 strong buy, and 0 strong sell recommendations.
*   **HP Inc (HPQ):** The outlook is more neutral, with most analysts recommending to hold. The latest data from April 2025 shows 4 buy, 13 hold, 1 sell, 2 strong buy, and 0 strong sell recommendations.
*   **Hewlett Packard Enterprise (HPE):** The sentiment is positive. As of April 2025, there are 7 buy, 8 hold, 0 sell, 4 strong buy, and 0 strong sell recommendations.
*   **Super Micro Computer Inc (SMCI):** The recommendations are mixed. The April 2025 data shows 8 buy, 8 hold, 2 sell, 2 strong buy, and 0 strong sell recommendations.
*   **NetApp Inc (NTAP):** The sentiment is leaning towards hold. In April 2025, there are 8 buy, 17 hold, 0 sell, 3 strong buy, and 0 strong sell recommendations.
*   **Pure Storage Inc - Class A (PSTG):** The recommendations are strongly positive. The latest data from April 2025 shows 13 buy, 6 hold, 1 sell, 8 strong buy, and 0 strong sell recommendations.
*   **Western Digital Corp (WDC):** The recommendations are strongly positive, with a majority of analysts suggesting a buy. In April 2025, there are 15 buy, 8 hold, 0 sell, 6 strong buy, and 0 strong sell recommendations.
*   **IONQ Inc (IONQ):** The recommendations are strongly positive. The latest data from April 2025 shows 7 buy, 2 hold, 0 sell, 2 strong buy, and 0 strong sell recommendations.

In [25]:
send_message('''How does the recent news say about Apple and the impact of tariffs? Over the past month.''')

#### Natural language response

Over the past month, news regarding Apple and the impact of tariffs has been varied. Initially, there were concerns about potential tariffs on electronics like smartphones and computers, which led to worries about increased prices and potential customer loyalty issues. Some analysts suggested that Apple could face an "existential" risk in China due to these factors.

However, there were also reports that the Trump administration had granted exclusions for certain electronics, including smartphones, from reciprocal tariffs, leading to a surge in Apple's stock price. This was seen as a positive development, providing some tariff relief and easing Wall Street concerns. Some analysts suggested that this exemption could be "semi-permanent."

Despite the tariff relief, some concerns remained about Apple's growth story and overexposure to tariff risks. There were also reports of declining iPhone sales in China and increased competition from local rivals.

Overall, the news suggests a complex and uncertain situation for Apple regarding tariffs, with potential benefits from exemptions but also ongoing risks and challenges related to trade tensions and competition in key markets.


# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>